In [1]:
# 使用方法
# $ python mp4_descripter.py [引数1] [引数2]
#
# 引数1: ファイル名
# 引数2: 内容を表示したいコンテナのインデックス

import sys
import binascii

leaf_list = ['fiel', 'mdat', 'rdrf', 'rmcd', 'rmcs', 'rmdr', 'rmqu', 'rmvc', 'wfex', 'cmvd', 'co64', 'dcom', 'elst', 'gmhd', 'hdlr', 'mdhd', 'smhd', 'stco', 'stsc', 'stsd', 'stss', 'stsz', 'stts', 'tkhd', 'vmhd']
parent_list = ['cmov', 'ctts', 'edts', 'esds', 'free', 'ftyp', 'iods', 'junk', 'mdia', 'minf', 'moov', 'mvhd', 'pict', 'pnot', 'rmda', 'rmra', 'skip', 'stbl', 'trak', 'uuid', 'wide']

In [2]:
def read_container(file, indent, index, end_offset, content_index):
    # read container size
    data = file.read(4)
    size = int.from_bytes(data, byteorder='big')
    if size == 0: return 0

    # read container type
    data = file.read(4)
    type = data.decode("utf-8") 
    is_leaf = (type in leaf_list)

    end = file.tell() + size - 8

    # when container size exceeds 2147483647
    if size == 1:
        file_pos = file.tell()
        data = file.read(8)
        size = int.from_bytes(data, byteorder='big')
        file.seek(file_pos, 0)

    if not is_leaf:
        file_pos = file.tell()
        file.seek(4, 1)
        next_type = ''
        try:
            next_type = file.read(4).decode("utf-8") 
        except Exception:
            next_type = 'unknown box name'
        file.seek(file_pos, 0)
        # unknown box is assumed as leaf
        if next_type not in leaf_list and next_type not in parent_list:
            is_leaf = True

    # make index string
    if index < 10:
        out_text = '0' + str(index) + ': '
    else:
        out_text = str(index) + ': '

    for i in range(indent): out_text += '   '

    if is_leaf:
        out_text += type
        out_text += '  -  '
        out_text += str(size)
        print(out_text)

        if index == int(content_index):
            # print binary data
            print('')
            print('offset:' + str(hex(file.tell() - 8)))
            data = file.read(size - 8)
            strarray = str(binascii.hexlify(data))[2:-1]
            outstr = ''
            for i in range(int(len(strarray) / 2)):
                if i % 8 == 0:
                    outstr = outstr + ' '
                if i % 16 == 0:
                    print(outstr)
                    outstr = ''
                temp = strarray[i*2:i*2+2]
                outstr = outstr + temp + ' '
            print(outstr)
            print('')
        else:
            # move to next container
            file.seek(size - 8, 1)
        return index + 1
    else:
        out_text += type
        print(out_text)
        if index == int(content_index):
            print('')
            print('offset:' + str(hex(file.tell() - 8)))
            print('no data')
            print('')
        index += 1
        while True:
            # move to children
            index = read_container(file, indent + 1, index, end, content_index)
            if file.tell() == end: break
        return index

In [ ]:
url = r'E:\ダウンロード\Akira.mp4'
file = open(url,'r+b')
count_index = 0
index = 1
index = read_container(file, 1, index, -1,count_index)
index = read_container(file, 1, index, -1,count_index)
file.close()

01:    ftyp  -  32
02:    free
03:       mdat  -  1938036201
04:       moov
05:          mvhd  -  108
06:          iods  -  24
07:          trak
08:             tkhd  -  92
09:             edts
10:                elst  -  28
11:             tref  -  20
12:             mdia
13:                mdhd  -  32
14:                hdlr  -  45
15:                minf
16:                   vmhd  -  20
17:                   dinf  -  36
18:                   stbl
19:                      stsd  -  173
20:                      stts  -  1406720
21:                      stss  -  8972
22:                      ctts  -  1325904
23:                      stsc  -  28
24:                      stsz  -  719204
25:                      stco  -  719200
26:          trak
27:             tkhd  -  92
28:             edts
29:                elst  -  28
30:             tref  -  20
31:             mdia
32:                mdhd  -  32
33:                hdlr  -  39
34:                minf
35:                   smhd  -  1